In [1]:
import glob
import numpy as np
import pandas as pd
import csv
import tensorflow as tf
from keras.layers import Dense, Activation, Flatten, Dropout, LSTM, Conv1D
from keras.models import Sequential, Model
from keras import regularizers, optimizers
import keras
import sklearn
from sklearn.model_selection import train_test_split

In [2]:
cough_df = pd.read_csv("cough_dataset.csv")
del cough_df['filename']
cough_df['label']="Cough"
no_cough_df = pd.read_csv("nocough_dataset.csv")
del no_cough_df['filename']
no_cough_df['label']="No Cough"
no_cough_df = no_cough_df.sample(frac=1) #Shuffle dataset to ensure no-cough 
no_cough_df = no_cough_df.head(len(cough_df)) #The two datasets should be the same length 

In [3]:
full_df = pd.concat([cough_df,no_cough_df])
features = full_df.columns[0:-1]
labels = pd.get_dummies(full_df['label']).values
"""
I chose to one-hot-encode the labels; originally I had assigned 0 to Covid and 1 to No Covid, but this made interpreting results incredibly difficult.
pd.get_dummies() turns the results into two columns - if the original value was "Covid," the row is [1,0]. Otherwise, the value is [0,1].
Intepreting model predictions becomes much easier; if a higher value is in the first column, the model has predicted Cough, otherwise, the model has predicted No Cough
"""
input_data = full_df[features].values

In [4]:
X_train, X_test, y_train, y_test =  train_test_split(input_data,labels,test_size=0.1)

In [50]:
"""
There isn't a whole lot of logic to this so far - the model needs improving. 
"""

def get_basic_model():
  model = Sequential()
  model.add(Conv1D(254, 3, padding='same',input_shape=(56,1),activation='relu'))
  model.add(Dense(136,kernel_regularizer='l1'))
  model.add(Dropout(.2))
  model.add(LSTM(105,return_sequences=True,dropout=0.2))
  model.add(Dropout(.2))
  model.add(Dense(20,kernel_regularizer='l1'))
  model.add(Flatten())
  model.add(Dense(2,activation="sigmoid")) #Output has two columns (Covid, Non-Covid). Sigmoid activation is also supposed to be best for class-based entropy

  model.compile(optimizer='rmsprop',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
  model.summary()
  return model

In [51]:
model = get_basic_model()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_13 (Conv1D)          (None, 56, 254)           1016      
                                                                 
 dense_33 (Dense)            (None, 56, 136)           34680     
                                                                 
 dropout_25 (Dropout)        (None, 56, 136)           0         
                                                                 
 lstm_18 (LSTM)              (None, 56, 105)           101640    
                                                                 
 dropout_26 (Dropout)        (None, 56, 105)           0         
                                                                 
 dense_34 (Dense)            (None, 56, 20)            2120      
                                                                 
 flatten_12 (Flatten)        (None, 1120)            

In [52]:
model.fit(X_train,y_train,epochs=20,validation_split=0.1)

Epoch 1/20
304/304 [==============================] - 27s 75ms/step - loss: 6.3824 - accuracy: 0.6515 - precision_11: 0.6491 - recall_11: 0.6428 - val_loss: 0.9125 - val_accuracy: 0.6222 - val_precision_11: 0.6206 - val_recall_11: 0.6481
Epoch 2/20
304/304 [==============================] - 18s 58ms/step - loss: 0.8602 - accuracy: 0.6691 - precision_11: 0.6641 - recall_11: 0.6858 - val_loss: 0.8076 - val_accuracy: 0.6907 - val_precision_11: 0.6823 - val_recall_11: 0.7019
Epoch 3/20
304/304 [==============================] - 19s 64ms/step - loss: 0.8105 - accuracy: 0.6924 - precision_11: 0.6849 - recall_11: 0.7062 - val_loss: 0.7811 - val_accuracy: 0.7028 - val_precision_11: 0.6980 - val_recall_11: 0.7148
Epoch 4/20
304/304 [==============================] - 17s 57ms/step - loss: 0.7886 - accuracy: 0.7048 - precision_11: 0.7049 - recall_11: 0.7046 - val_loss: 0.7539 - val_accuracy: 0.7037 - val_precision_11: 0.7063 - val_recall_11: 0.6926
Epoch 5/20
304/304 [============================

In [53]:
"""
Want to make sure that predictions are correct! More importantly, want to make sure that coughs are correctly classified as coughs. 
Overall accuracy isn't the best metric - we care more about f-score (precision/recall). 
Precision: What was correctly classified as cough?  True Coughs/(True Coughs + False Coughs)
Recall: Of what was actually a cough, what percentage was correctly classified?  True Coughs/(True Coughs + False Non-Coughs )
"""

predictions = model.predict(X_test)
reals = []
preds = []
for i in range(len(predictions)):
    reals.append(np.argmax(y_test[i]))
    preds.append(np.argmax(predictions[i]))
sklearn.metrics.precision_recall_fscore_support(reals,preds) #First column is cough, second column is non-cough

38/38 [==============================] - 1s 23ms/step


(array([0.83903421, 0.74537696]),
 array([0.69966443, 0.86754967]),
 array([0.76303751, 0.80183627]),
 array([596, 604], dtype=int64))